<a href="https://colab.research.google.com/github/LDwise/autoWiseman/blob/main/auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# coding=utf-8
from re import findall
from time import time
from google.colab import output
from IPython.display import display,HTML
try:
	from selenium import webdriver
except:
	# 清除輸出Log
	print('Install package...\n下載所需插件以便執行...')
	!apt -q -qq update
	!apt -q -qq install chromium-chromedriver
	!pip -q install selenium
	from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
# 設置Chrome啓動設置
options = webdriver.ChromeOptions()
# 無界面運行
options.add_argument('--headless')
# 以最高权限运行
options.add_argument('--no-sandbox')
# 忽略警告\錯誤\異常
# options.add_argument('log-level=3')
# log-level: 
# Sets the minimum log level.
# Valid values are from 0 to 3: 
#		 INFO = 0, 
#		 WARNING = 1, 
#		 LOG_ERROR = 2, 
#		 LOG_FATAL = 3.
# default is 0.
# 避免崩潰
options.add_argument('--disable-dev-shm-usage')
# 開啓Chrome並加入基礎設置
driver = webdriver.Chrome(options=options)
# 設定等待元素時間上限(單位秒)
wait = WebDriverWait(driver,10)
# 清除輸出内容
output.clear()
#@markdown # ←點擊▶按鈕開始運行刷分
#@markdown <img src="https://lms.wiseman.com.hk/lms/user/images/login_wiseman_logo.png" width="100px"><br>
#@markdown ## Welcome to autoWiseman program.
#@markdown 針對[Wiseman英文網站](https://lms.wiseman.com.hk/lms/user/logout.do)的綫上刷分程序\
#@markdown 100分滿分為基準,90%成功率
#@markdown ### 請先在在下方輸入個人賬號及密碼
username = "tiaccwhf.5a29" #@param {type:"string"}
password = "2020" #@param {type:"string"}
#@markdown ### 選擇相應課程類型
lessons_type = "Wiseman(EB level 1)" #@param ["Wiseman(EB level 1)", "Fun and Friends(Foundation)"]
#@markdown ### 設定每課程的分數
score = 100 #@param {type:"slider", min:0, max:100, step:1}
########################################################################################################################
if lessons_type == "Fun and Friends(Foundation)":
	print('登入Fun and Friends',end='\r')
	driver.get('https://fnfclub.hk/lms/user/login.do?username=%s&password=%s'%(username,password))
	# Lesssons list
	driver.get('https://fnfclub.hk/lms/user/secure/course/selector/do/lessons.shtml')
	# link loop
	wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="effect_session"]/table/tbody/tr/td[4]')))
	display(HTML(
		# CSS style
		driver.find_element_by_xpath('//head').get_attribute('innerHTML').replace('href="','href="https://fnfclub.hk')+
		# Student_Score
		driver.find_element_by_xpath('//div[@class="effect_session"]').get_attribute('innerHTML').replace('href="','href="https://fnfclub.hk')
		))
	# uncomplete lessons
	uncomplete = driver.find_element_by_xpath('//div[@class="effect_session"]/table/tbody/tr/td[4][contains(., "-")]')
	while uncomplete:
		# check if link present
		try:
			a = uncomplete.find_element_by_xpath('./..//a[contains(@href,"/lms/user/secure/course/selector/do/selectLesson.do")]')
		except:
			continue
		print('Title:%s(%s)' % (a.get_attribute('textContent'), a.get_attribute('href')))
		driver.get(a.get_attribute('href'))
		wait.until(EC.presence_of_element_located((By.XPATH, '//iframe[@id="course"]')))
		tm = time()
		while time()-tm <= 0.5:
			print('Waiting:%f\'s' % (time()-tm), end='\r')
		# 刷新頁面
		driver.refresh()
		wait.until(EC.presence_of_element_located((By.XPATH, '//iframe[@id="course"]')))
		tm = time()
		while time()-tm <= 0.5:
			print('Waiting:%f\'s' % (time()-tm), end='\r')
		# Commit.do
		driver.execute_script('''
		API.cmiData["cmi.core.score.raw"]=%d;
		API.cmiData["cmi.core.lesson_status"]="completed";
		var formData = new FormData();
		formData.set('token', API.token);
		formData.set('data', API.cmiData.toJSONString());
		navigator.sendBeacon("commit.do", formData);
		navigator.sendBeacon=function(){return false;};
		'''%(score))
		# Lesssons list
		driver.get('https://fnfclub.hk/lms/user/secure/course/selector/do/lessons.shtml')
		# link loop
		wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="effect_session"]/table/tbody/tr/td[4]')))
		# uncomplete lessons
		uncomplete = driver.find_element_by_xpath('//div[@class="effect_session"]/table/tbody/tr/td[4][contains(., "-")]')
	# 登出Fun and Friends
	driver.get('https://fnfclub.hk/lms/user/logout.do')
#################################################################################################################
elif lessons_type == "Wiseman(EB level 1)":
	print('登入Wiseman',end='\r')
	driver.get('https://lms.wiseman.com.hk/lms/user/login.do?username=%s&password=%s'%(username,password))
	display(HTML(
		# CSS style
		driver.find_element_by_xpath('//head').get_attribute('innerHTML').replace('href="','href="https://lms.wiseman.com.hk')+
		# Profile → Student_Name
		driver.find_element_by_xpath('//div[@class="profile-text"]/a[@class="dropdown-toggle link u-dropdown"]').get_attribute('innerHTML').replace('href="','href="https://lms.wiseman.com.hk')+#//*[@id="main-wrapper"]/aside/div/div[1]/div/div[2]/a
		# Student_Score
		driver.find_element_by_xpath('//div[@class="card-body"]/div[@class="text-right"]').get_attribute('innerHTML')#//*[@id="main-wrapper"]/div/div/div[2]/div[1]/div/div/div/h1
	))
	# Lesssons list
	driver.get('https://lms.wiseman.com.hk/lms/user/secure/course/eb/select_theme/lessons.shtml')
	# link loop
	wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="table-responsive"]/table[@class="table table-striped"]/tbody/tr')))
	# uncomplete lessons
	uncomplete = driver.find_element_by_xpath('//div[@class="table-responsive"]/table[@class="table table-striped"]/tbody/tr/td[@class="align-middle text-center"][contains(., "-")]')
	while uncomplete:
		# check if link present
		try:
			a = uncomplete.find_element_by_xpath('./..//a[contains(@href,"/lms/user/secure/course/eb/select_theme/selectLesson")]')
		except:
			continue
		print('Title:%s(%s)' % (a.get_attribute('textContent'), a.get_attribute('href')))
		driver.get(a.get_attribute('href'))
		wait.until(EC.presence_of_element_located((By.XPATH, '//iframe[@id="course"]')))
		tm = time()
		while time()-tm <= 0.5:
			print('Waiting:%f\'s' % (time()-tm), end='\r')
		# if Changllenage present: click button
		try:
			driver.execute_script('''
			// 100% page
			course=document.querySelector('iframe[id="course"]').contentWindow;
			// Changllenage
			course.document.querySelector('label[id="select-group-1"]').click();
			// Start
			course.document.querySelector('button[data-text="START"]').click();
			''')
		except:
			print('無需點擊Challenge', end='\r')
		# 刷新頁面
		driver.refresh()
		wait.until(EC.presence_of_element_located((By.XPATH, '//iframe[@id="course"]')))
		tm = time()
		while time()-tm <= 0.5:
			print('Waiting:%f\'s' % (time()-tm), end='\r')
		# Commit.do
		driver.execute_script('''
		API.cmiData["cmi.core.score.raw"]=%d;
		API.cmiData["cmi.core.lesson_status"]="completed";
		var formData = new FormData();
		formData.set('token', API.token);
		formData.set('data', API.cmiData.toJSONString());
		navigator.sendBeacon("commit.do", formData);
		navigator.sendBeacon=function(){return false;};
		'''%(score))
		# Lesssons list
		driver.get('https://lms.wiseman.com.hk/lms/user/secure/course/eb/select_theme/lessons.shtml')
		# link loop
		wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="table-responsive"]/table[@class="table table-striped"]/tbody/tr')))
		# uncomplete lessons
		uncomplete = driver.find_element_by_xpath('//div[@class="table-responsive"]/table[@class="table table-striped"]/tbody/tr/td[@class="align-middle text-center"][contains(., "-")]')
	# 進入分數頁面
	driver.get('https://lms.wiseman.com.hk/lms/user/secure/course/eb/select_theme/dashboard.shtml')
	wait.until(EC.presence_of_element_located((By.XPATH,'//div[@class="card-body"]/div[@class="text-right"]')))
	display(HTML(
		# Profile → Student_Name
		driver.find_element_by_xpath('//div[@class="profile-text"]/a[@class="dropdown-toggle link u-dropdown"]').get_attribute('innerHTML').replace('href="','href="https://lms.wiseman.com.hk')+#//*[@id="main-wrapper"]/aside/div/div[1]/div/div[2]/a
		# Student_Score
		driver.find_element_by_xpath('//div[@class="card-body"]/div[@class="text-right"]').get_attribute('innerHTML')+#//*[@id="main-wrapper"]/div/div/div[2]/div[1]/div/div/div/h1
		'<h1>All Finished</h1>'
	))
	# 登出Wiseman
	driver.get('https://lms.wiseman.com.hk/lms/user/logout.do')
driver.quit()